<a href="https://colab.research.google.com/github/LevWilliams/PoliticalClassificationPOSTBERT/blob/master/PNClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modules and Libraries

In [18]:
## importing packages
import gc
import os
import random
!pip install transformers
import transformers
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K

from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from transformers import AutoTokenizer, TFAutoModel

print(f"TensorFlow version: {tf.__version__}")
print(f"Transformers version: {transformers.__version__}")

warnings.filterwarnings("ignore")

TensorFlow version: 2.3.0
Transformers version: 3.0.2


In [12]:
#tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

# instantiate a distribution strategy
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.95.145.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.95.145.42:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Uploading Data

In [2]:
PolySocMed = pd.read_csv("political_social_media.csv", encoding = 'iso-8859-1')

In [3]:
PolySocMed.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,message:confidence,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,1.0,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,1.0,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,1.0,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,1.0,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [4]:
TrainAndValDat = PolySocMed[['text','bias']]
TrainAndValDat.head()

,text,bias
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...,partisan
1,VIDEO - #Obamacare: Full of Higher Costs and ...,partisan
2,Please join me today in remembering our fallen...,neutral
3,RT @SenatorLeahy: 1st step toward Senate debat...,neutral
4,.@amazon delivery #drones show need to update ...,partisan


In [5]:
TrainAndValDat['bias']

0       partisan
1       partisan
2        neutral
3        neutral
4       partisan
          ...   
4995    partisan
4996    partisan
4997     neutral
4998     neutral
4999     neutral
Name: bias, Length: 5000, dtype: object

Tokenizing the data

In [6]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("albert-xxlarge-v2")
			
model = AutoModelWithLMHead.from_pretrained("albert-xxlarge-v2")
			

In [7]:
#returns tokenized data
def tokenize_sentences(sentences, tokenizer, max_seq_len = 100):
    tokenized_sentences = []
    for sentence in sentences:
        tokenized_sentence = tokenizer.encode(sentence, add_special_tokens = True, max_length = max_seq_len,truncation = True)
        tokenized_sentences.append(tokenized_sentence)
    return tokenized_sentences

def preprocessPoliticalData():
    print("Loading data...")
    x_text = TrainAndValDat['text']
    y = TrainAndValDat['bias']
    tokenizer =  AutoTokenizer.from_pretrained("albert-xxlarge-v2", do_lowercase=True)

    ydict = {
        'neutral':0,
        'partisan':1
    }

    x_train = tokenize_sentences(x_text, tokenizer)
    x = np.array(list(tf.keras.preprocessing.sequence.pad_sequences(x_train, 100, padding='post', truncating='post')))

    shuffle_indices = np.random.permutation(np.arange(len(y)))
    x_train = x[shuffle_indices]
    y_train = y[shuffle_indices].apply(lambda x: ydict[x])

    del x, y
    return x_train, y_train

In [9]:

x_train, y_train = preprocessPoliticalData()

Loading data...


In [10]:
y_train

3714    0
3948    0
2738    1
3058    1
2798    0
       ..
2013    0
3124    0
2823    0
3398    0
3665    0
Name: bias, Length: 5000, dtype: int64

Model

In [19]:
## building model
def build_model(model_name, max_len):
    """
    Building the Deep Learning architecture
    """
    # defining encoded inputs
    input_ids = Input(shape = (max_len,), dtype = tf.int32, name = "input_ids")
    
    # defining transformer model embeddings
    transformer_model = TFAutoModel.from_pretrained(model_name)
    transformer_embeddings = transformer_model(input_ids)[0]
    
    # defining output layer
    output_values = Dense(2, activation = "softmax")(transformer_embeddings[:, 0, :])

    # defining model
    model = Model(inputs = input_ids, outputs = output_values)
    opt = Adam(learning_rate = 1e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
    metrics = metrics

    model.compile(optimizer = opt, loss = loss)

    return model

In [25]:
def run_model():
      # K-fold on comment and label
    kf = KFold(n_splits = 10, shuffle = True, random_state = 2)
        # initializing predictions
    preds_oof = np.zeros((x_train.shape[0], 3))
    acc_oof = []


    for (fold, (train_index, valid_index)) in enumerate(kf.split(x_train, y_train)):
        # initializing TPU
        K.clear_session()
        with tpu_strategy.scope():
            model = build_model("albert-xxlarge-v2", 100)
            if fold == 0:
                print(model.summary())
                print("\n")
        
        print("#" * 19)
        print(f"##### Fold: {fold + 1} #####")
        print("#" * 19)

        # splitting data into training and validation
        X_train = x_train.iloc[train_index]
        X_valid = x_train.iloc[valid_index]

        Y_train = X_train.label.values
        Y_valid = X_valid.label.values

        # creating TF Dataset
        ds_train = get_tf_dataset(X_train, y_train, tf.data.experimental.AUTOTUNE, repeat = True, shuffle = True, batch_size = 32)
        ds_valid = get_tf_dataset(X_valid, y_valid, tf.data.experimental.AUTOTUNE, batch_size = 32)

        n_train = X_train.shape[0]

        # saving model at best accuracy epoch
        sv = tf.keras.callbacks.ModelCheckpoint(
            "model.h5",
            monitor = "val_sparse_categorical_accuracy",
            verbose = 0,
            save_best_only = True,
            save_weights_only = True,
            mode = "max",
            save_freq = "epoch"
        )

        # training model
        model_history = model.fit(
            ds_train,
            epochs = 20,
            callbacks = [sv],
            steps_per_epoch = 100,
            validation_data = ds_valid,
            verbose = verbose
        )

        print("\nValidating")

        # scoring validation data
        model.load_weights("model.h5")
        ds_valid = get_tf_dataset(X_valid_encoded, -1, tf.data.experimental.AUTOTUNE, labelled = False, batch_size = 32)

        preds_valid = model.predict(ds_valid, verbose = verbose)
        acc = accuracy_score(y_valid, np.argmax(preds_valid, axis = 1))

        preds_oof[valid_index] = preds_valid
        acc_oof.append(acc)

        print("\nInferencing")

    # overall CV score and standard deviation
    print(f"\nCV Mean Accuracy: {round(np.mean(acc_oof), 4)}")
    print(f"CV StdDev Accuracy: {round(np.std(acc_oof), 4)}\n")

    return preds_oof


In [26]:
hiw = run_model()

UnboundLocalError: ignored